# Introduction
This repository contains the implementation of a 2D NERF (Neural Radiance Fields) model for image rendering using PyTorch in a Jupyter Notebook format. The model is designed to render images using a 9-layer Multi-Layer Perceptron (MLP) and sinusoidal position encoding (PE).




In [ ]:
#importing needed liberaries
import torch 
import torch.nn as nn
from tqdm.notebook import tqdm
import numpy as np
import cv2
import random
import torch.functional as F

# Model Configuration
The model is configured with the following hyperparameters:

In [ ]:
#configuration Hyperparameters 
image_size = 100
n_layers =9
layer_units =256
epochs =200
lr =0.00001
encoding_degree = 5
batch_size = 32
up_scale = 4
momentum =0.5
device ='cuda'

In [ ]:
class SRM(nn.Module):
    def __init__(self,n_layers,layer_units,encoding_degree=5,sinsuidal_activation=False,use_position_encoding=True):
        super(SRM,self).__init__()
        self.encoding_degree = encoding_degree
        self.n_layers=n_layers
        self.use_position_encoding =use_position_encoding
        self.sinsuidal_activation=sinsuidal_activation
        self.layer_units =layer_units
        if use_position_encoding :
            self.input = nn.Linear(4*encoding_degree,layer_units)
        else:
            self.input = nn.Linear(2,layer_units)
        self.layers =nn.ModuleList([nn.Linear(self.layer_units,self.layer_units) for _ in range(self.n_layers) ])
        self.output =nn.Linear(self.layer_units,3) #RGB
        
        self.activation = nn.ReLU(inplace=True)
        
        self.out_act =nn.Softmax()

    def forward(self,x):
        if self.use_position_encoding:

            l =[]
            for i in range(self.encoding_degree):
                l+=[torch.sin(2**i*np.pi*x[:,0].unsqueeze(1)),torch.cos(2**i*np.pi*x[:,0].unsqueeze(1))]
                l+= [torch.sin(2**i*np.pi*x[:,1].unsqueeze(1)),torch.cos(2**i*np.pi*x[:,1].unsqueeze(1))] 
            #for i in range(self.encoding_degree):
            #    pass

            embeded = torch.concat(l,dim=-1)
            z = self.input(embeded)
        else:
            z = self.input(x)
        for l in self.layers:
            if self.sinsuidal_activation:
                z =torch.sin(l(z))
            else:
                z =self.activation(l(z)) 
        return self.output(z)

In [ ]:
model =SRM(n_layers,layer_units,encoding_degree,sinsuidal_activation=True,use_position_encoding=True)
x = torch.randn([10,2])

model(x).shape

torch.Size([10, 3])

In [ ]:
!wget -O image.jpg https://unsplash.com/photos/J7fxkhtOqt0/download?force=true&w=640 
!mogrify -resize 100x100! image.jpg

In [ ]:
def generator(batch):
    image = cv2.imread('image.jpg')
    x = []
    y = []
    rr =[s for s in range(image_size)]
    random.shuffle(rr)
    while True:
        for i in rr:
            for j in rr:
                x.append([i,j])
                y.append(image[i,j])
                if len(x)==batch:
                    yield (np.array(x,dtype=np.float32)/image_size,np.array(y,dtype=np.float32))
                    x=[]
                    y=[]

In [ ]:
criterion =nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr=lr,momentum=momentum)


In [ ]:
import matplotlib.pyplot as plt
def display(model):
    x,y =np.meshgrid(np.linspace(0,image_size,image_size),np.linspace(0,image_size,image_size))
    indexs = np.concatenate([y.reshape([image_size*image_size,1]),x.reshape([image_size*image_size,1])],axis=1)
    ys =model(torch.from_numpy(indexs.astype(np.float32)/image_size))
    image_g =np.reshape(ys.detach().numpy(),[image_size,image_size,3])
    
    img =cv2.imread('image.jpg')
    img2 =cv2.imread('image2.jpg')

    plt.figure(figsize=(16,3))
    plt.subplot(1,4,1)
    plt.imshow(img)
    plt.title('Original Image.')
    plt.subplot(1,4,2)
    plt.imshow(image_g.astype(np.uint8))
    plt.title('Memorized Image by MLP Model.')
    plt.show()

In [ ]:
g =generator(batch_size)
t =tqdm(total=epochs)

for epoch in range(epochs):
    losses =0
    model.to(device)
    for i in range(image_size**2//batch_size):
        x,y = next(g)
        x = torch.from_numpy(x).to(device)
        y = torch.from_numpy(y).to(device)
        y1 = model(x)
        loss = criterion(y1,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    losses=loss.cpu().detach().numpy()
    t.update()
    
        #t.display(f"Epoch : {i},loss :{sum(losses)/(i+1)} , Done : {i/image_size**2//batch_size}")
    #if i%10000:
    t.close()
    if epoch%50==0:
        display(model.cpu() )
    print(f"Epoch : {epoch},loss :{losses} ")#

if needed

In [ ]:
!apt-get install imagemagick